# Basic simulation analysis

In this notebook you will run some basic analyses and plots on a simulation.

The simulation you are going to analyse has the following parameters:
- all neurons are simulated
- duration of 1000 ms
- projections (Schaffer collaterals) included
- spontaneous synaptic activity (minis) at 0.01 Hz

---

Import some python packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# package to work with the circuit
from bluepysnap import Simulation
from bluepysnap.bbp import Cell

Fix the seed

In [ ]:
np.random.seed(12)

Get the simulation data and create the simulation object

In [ ]:
sim = Simulation("/home/data-bbp/20191017/simulation_config.json")

Get the global spikes report and the specific hippocampus one

In [ ]:
spikes = sim.spikes
hippocampus_spikes = spikes["hippocampus_neurons"]

Get the hippocampus neurons which spiked and check the properties of the spiking neurons

In [ ]:
hippocampus_spikes.nodes.get(group=hippocampus_spikes.node_ids).head()

Filter the report on SP_PC cells from the mc2 column only

In [ ]:
filtered_spikes = spikes.filter(group={Cell.MTYPE: 'SP_PC', 'region': {'$regex': "mc2.*"}})

In [ ]:
filtered_spikes.report.head()

In [ ]:
# Hack : cell we need to remove with a better simulation
times = filtered_spikes.report.index.values
filtered_spikes.report.loc[times.max() + 0.15] = [3311, "hippocampus_neurons"]
times = filtered_spikes.report.index.values
filtered_spikes.report.loc[times.max() + 0.15] = [602, "hippocampus_neurons"]

Spiking neurons and number of spikes per neurons during the simulation for the filtered cells

In [ ]:
node_ids, nb_spikes = np.unique(filtered_spikes.report["ids"], return_counts=True)

Get full set of properties for neurons which spikes more than once

In [ ]:
hippocampus_spikes.nodes.get(group=node_ids[nb_spikes > 1])

Get full set of  properties for neurons which spikes only once

In [ ]:
hippocampus_spikes.nodes.get(group=node_ids[nb_spikes == 1]).head()

### Raster plot

In [ ]:
filtered_spikes.raster() ;

### Instantaneous frequency plot

In [ ]:
filtered_spikes.firing_rate_histogram() ;

### Inter-spike interval (ISI) distribution

In [ ]:
filtered_spikes.isi() ;

### ISI frequency distribution

In [ ]:
filtered_spikes.isi(use_frequency=True) ;

### Mean frequency distribution

In [ ]:
# Function to get mean frequencies for each neuron from the report
def get_mean_frequencies():
    gb = filtered_spikes.report.groupby(["ids", "population"])
    res = []
    for _, node_spikes in gb:
        time_differences = np.diff(node_spikes.index.to_numpy())
        if len(time_differences) != 0:
            res.append(1000/time_differences.mean())
    return np.asarray(res)

In [ ]:
# Function to plot the mean frequency distribution
def plot_spike_frequency(mean_frequencies, ax=None):
    
    if ax is None:
        ax = plt.gca()
        ax.set_xlabel('Frequency [Hz]')
        ax.set_ylabel('# neurons')

    fig = plt.gcf()
    
    mean = mean_frequencies.mean()
    ax.hist(mean_frequencies)
    ax.axvline(x=mean, color='red')
    
    text = f'{mean:2.8f}'
    limits =  ax.get_xlim()
    mean_text_pos = (limits[1] - mean)/(limits[1] - limits[0]) + 0.05
    
    fig.text(mean_text_pos, 0.80, text, color='red', fontsize=12)
    
    return ax

In [ ]:
plot_spike_frequency(get_mean_frequencies()) ;

### Visualization

In [ ]:
from IPython.display import HTML
anim, ax = filtered_spikes.firing_animation()
plt.close() # need this to avoid the first frame to appear in a notebook
HTML(anim.to_html5_video())

### Voltage traces

Show the different available reports for this simulation

In [ ]:
print(sim.reports)

Reports can be massive we select just few node_ids

In [ ]:
hippocampus_soma_report = sim.reports["soma_report"]["hippocampus_neurons"]
node_ids = hippocampus_soma_report.nodes.ids(group={Cell.MTYPE: 'SP_PC', 'region': {'$regex': "mc2.*"}}, sample=15)

In [ ]:
filtered_report_soma = sim.reports["soma_report"].filter(group=node_ids)
filtered_report_soma.report.head()

Plot traces from selected cells

In [ ]:
def multi_traces(nids):
    '''
    Function to plot multiple voltage traces given the ids of the nodes
    '''
    etypes = hippocampus_soma_report.nodes.get(node_ids, Cell.ETYPE)
    fig, axs = plt.subplots(nrows=len(nids), sharex=True, sharey=False, figsize=(6, 1.5 * len(nids)))
    for ax, node_id, etype in zip(axs, nids, etypes):
        ax.ticklabel_format(useOffset=False, style='plain')
        ax.set_ylabel("%d\n%s" % (node_id, etype))
        cdata = filtered_report_soma.report.loc[:, ("hippocampus_neurons", node_id)]
        ax.plot(np.array(cdata.index), cdata.values)
        ax.yaxis.set_label_position("right")
        ax.set_ylabel("a%d\n%s" % (node_id, etype))
        ax.grid()
    axs[-1].set_xlabel('Time [ms]')
    fig.text(0.02, 0.5, 'Voltage [mV]', va='center', rotation='vertical')

In [ ]:
multi_traces(node_ids[:5])

Plot all traces

In [ ]:
filtered_report_soma.trace(plot_type="all") ;

Plot an average trace

In [ ]:
# sample traces mean
filtered_report_soma.trace(plot_type="mean") ;

### Exercise #1
Description

In [ ]:
# This is to generate the answers to paste in the submission box below.
# After you defined the variables with your answers, run this cell and the next cell, and copy-paste the output into the box below
print(json.dumps(dict([("ans_1a", ans_1a),
                       ("ans_1b", ans_1b),
                       ("ans_2a", ans_2a),
                       ("ans_2b", ans_2b)])))

In [ ]:
!pip -q install -i https://bbpteam.epfl.ch/repository/devpi/simple/ single-cell-mooc-client==0.0.5 
import single_cell_mooc_client as sc_mc
s = sc_mc.Submission(hideToken=True)

In [ ]:
s.show_submission(ROLLBACK, TOKEN)